# Prepare wines from a dirty list

Load a CSV of wines (aka winelist) and clean the data to get it ready to run it through the matching algorithm.


In [163]:
import pandas as pd

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load dirty winelist file and perform cleaning


In [164]:
wines_original = pd.read_csv('v1-cleaned.csv')
print(f'Total rows: {wines_original.shape[0]}')
print()

wines_original.head()


Total rows: 928



,external_id,name_one,name_two,size,winery_name,country,vintage,currency,price,quantity,Unnamed: 10
0,1,Demi: 375 ml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Champagne,Rosé,375 ml,Ruinart,FRA,NaN,€,"€ 85,00",23,NaN
2,3,Champagne,Blanc de Blanc,375 ml,Ruinart,FRA,NaN,€,"€ 80,00",19,NaN
3,4,Venegazzù della Casa,NaN,375 ml,Loredan e Gasperin,ITA,17,€,"€ 25,00",NaN,NaN
4,5,Santa Maddalena,NaN,375 ml,Cantina di Bolzano,ITA,21,€,"€ 15,00",NaN,NaN


### Note: This part is always CUSTOM to the client


 - Merge `name_one` and `name_two`.
 - drop grappe e birre.

Other fields:
 - `external_id`: OK
 - `name`: merge name_one + name_two
 - `winery_name`: drop nan rows
 - `type`: missing
 - `storage_area`: missing
 - `size`: drop "al bicchiere" rows, to parse
 - `vintage`: to parse
 - `price`: to parse
 - `info`: missing
 - `quantity`: to parse
 - `internal_notes`: missing
 - `country`: extra field, is already in DB



#### Name

In [165]:
wines = wines_original.copy()
rows_to_drop = pd.DataFrame(columns=list(wines.columns)+['reason'])

def add_to_drop(condition, reason=''):
    to_drop = wines_original[condition]
    # Only keep rows that are not already in dropped_rows
    to_drop = to_drop[~to_drop.index.isin(rows_to_drop.index)]
    to_drop['reason'] = reason
    return pd.concat([rows_to_drop, to_drop])

import re

# external id ok

# merge name_one and name_two into name
wines['name'] = wines['name_one'].fillna('') + ' ' + wines['name_two'].fillna('')
wines['name'] = wines['name'].apply(lambda x: re.sub(r'\s+', ' ', x).strip().lower())

wines = wines.drop(columns=['name_one', 'name_two'])

In [166]:
rows_to_drop = add_to_drop(wines['name']=='', 'name is empty')

#### Drop grappe and beer

In [167]:
# drop beers and grappe
display(wines.loc[range(834, len(wines))])
rows_to_drop = add_to_drop(wines.index >= 834, reason='grappe or beer')
rows_to_drop

,external_id,size,winery_name,country,vintage,currency,price,quantity,Unnamed: 10,name
834,835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,birre
835,836,NaN,Moretti,NaN,NaN,€,"€ 15,00",NaN,NaN,grand cru
836,837,NaN,Aynger,NaN,NaN,€,"€ 6,00",NaN,NaN,weissen
837,838,NaN,Dolomiti,NaN,NaN,€,"€ 6,00",NaN,NaN,chiara 4.9
838,839,NaN,Birra di Parma,NaN,NaN,€,"€ 8,00",NaN,NaN,bionda
...,...,...,...,...,...,...,...,...,...,...
923,924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,saké
924,925,al bicchiere,NaN,NaN,NaN,€,NaN,NaN,NaN,gjkon
925,926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vermouth
926,927,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,vermouth tenuta fertuna


,external_id,name_one,name_two,size,winery_name,country,vintage,currency,price,quantity,Unnamed: 10,reason
14,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,name is empty
102,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,name is empty
138,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,name is empty
176,177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,name is empty
195,196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,name is empty
...,...,...,...,...,...,...,...,...,...,...,...,...
923,924,Saké,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,grappe or beer
924,925,Gjkon,NaN,al bicchiere,NaN,NaN,NaN,€,NaN,NaN,NaN,grappe or beer
925,926,Vermouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,grappe or beer
926,927,Vermouth Tenuta Fertuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,grappe or beer


#### winery_name

In [168]:
# winery_name: substitute nan with empty string
wines_original['winery_name'].unique()[:3]

array([nan, 'Ruinart', 'Loredan e Gasperin'], dtype=object)

In [169]:
rows_to_drop = add_to_drop(wines['winery_name'].isnull(), 'winery_name is null')
wines['winery_name'].unique()[:3]

array([nan, 'Ruinart', 'Loredan e Gasperin'], dtype=object)

#### type
#### storage_area

In [170]:
# type: missing
wines['type'] = ''

# storage_area: missing
wines['storage_area'] = ''

#### size

In [171]:
# `size`: drop "al bicchiere" rows
wines_original['size'].unique()

array([nan, '375 ml', 'Magnum', 'Jéroboam', '750 ml', '500 ml',
       'al bicchiere'], dtype=object)

In [172]:
sizes = {
    '375 ml': 'HALF_BOTTLE',
    '500 ml': 'HALF_LITER',
    '750 ml': 'BOTTLE',
    'Magnum': 'MAGNUM',
    'Jéroboam': 'JEROBOAM',
    'al bicchiere': 'al bicchiere'
}

wines['size'] = wines_original['size'].apply(lambda x: '750 ml' if pd.isna(x) else x)
wines['size'] = wines['size'].map(sizes)
print(wines['size'].unique())

rows_to_drop = add_to_drop(wines_original['size'] == 'al bicchiere', 'size is "al bicchiere"')

['BOTTLE' 'HALF_BOTTLE' 'MAGNUM' 'JEROBOAM' 'HALF_LITER' 'al bicchiere']


In [173]:
rows_to_drop.loc[rows_to_drop['reason'] == 'size is "al bicchiere"']

,external_id,name_one,name_two,size,winery_name,country,vintage,currency,price,quantity,Unnamed: 10,reason


#### vintage

In [174]:
# vintage: to parse
# set 0 vintage year to None and NaN vintage year to None
wines['vintage'] = wines_original['vintage'].apply(lambda x: None if pd.isnull(x) or pd.isna(x) else re.sub('\D', '', x))
print(wines['vintage'].unique())

# correct vintage and split vintages with multiple years
# later I am gonna use them to create multiple rows
wines['vintage'] = wines['vintage'].apply(lambda x: ('2021', '2022') if x == '2122' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('2019', '2020') if x == '1920' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('1979', '2019') if x == '19792019' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('2020', '2021') if x == '2021' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('2017', '2019', '2020') if x == '171920' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('2017', '2019') if x == '1719' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('2017', '2020') if x == '1720' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('2018', '2021') if x == '1821' else x)
wines['vintage'] = wines['vintage'].apply(lambda x: ('2013', '2019') if x == '1319' else x)

print(wines['vintage'].unique())

def process_year(year):
    if year is None or pd.isna(year):
        return year
    if type(year) == tuple:
        return tuple([int(y) for y in year])
    if int(year) < 24:
        return 2000 + int(year)
    if len(year) == 2 and int(year) >= 24:
        return 1900 + int(year)
    return int(year)

wines['vintage'] = wines['vintage'].apply(process_year)
print(wines['vintage'].unique())

[None '17' '21' '2122' '1920' '20' '19' '04' '8' '14' '10' '2008' '12'
 '13' '2005' '16' '18' '6' '19792019' '22' '23' '5' '68' '09' '11' '15'
 '06' '171920' '1975' '7' '2' '1719' '1720' '1821' '2016' '73' '2021' '9'
 '3' '97' '1' '95' '96' '98' '90' '99']
[None '17' '21' ('2021', '2022') ('2019', '2020') '20' '19' '04' '8' '14'
 '10' '2008' '12' '13' '2005' '16' '18' '6' ('1979', '2019') '22' '23' '5'
 '68' '09' '11' '15' '06' ('2017', '2019', '2020') '1975' '7' '2'
 ('2017', '2019') ('2017', '2020') ('2018', '2021') '2016' '73'
 ('2020', '2021') '9' '3' '97' '1' '95' '96' '98' '90' '99']
[None 2017 2021 (2021, 2022) (2019, 2020) 2020 2019 2004 2008 2014 2010
 2012 2013 2005 2016 2018 2006 (1979, 2019) 2022 2023 1968 2009 2011 2015
 (2017, 2019, 2020) 1975 2007 2002 (2017, 2019) (2017, 2020) (2018, 2021)
 1973 (2020, 2021) 2003 1997 2001 1995 1996 1998 1990 1999]


In [175]:
wines.loc[wines['vintage']==(1979, 2019)]

,external_id,size,winery_name,country,vintage,currency,price,quantity,Unnamed: 10,name,type,storage_area
89,90,BOTTLE,Loredan Gasparini,ITA,"(1979, 2019)",€,"€ 45,00",NaN,NaN,spumante extra brut,,


#### price

In [176]:
# price: to parse, must be in cents

wines_original['price'].unique()

array([nan, '€ 85,00', '€ 80,00', '€ 25,00', '€ 15,00', '€ 24,00',
       '€ 18,00', '€ 23,00', '€ 130,00', '€ 27,00', '€ 42,00', '€ 16,00',
       '€ 95,00', '€ 110,00', '€ 230,00', '€ 160,00', '€ 850,00',
       '€ 400,00', '€ 380,00', '€ 90,00', '€ 450,00', '€ 900,00',
       '€ 210,00', '€ 220,00', '€ 420,00', '€ 140,00', '€ 320,00',
       '€ 240,00', '€ 190,00', '€ 150,00', '€ 50,00', '€ 105,00',
       '€ 54,00', '€ 100,00', '€ 60,00', '€ 65,00', '€ 250,00', '€ 38,00',
       '€ 45,00', '€ 48,00', '€ 30,00', '€ 28,00', '€ 70,00', '€ 33,00',
       '€ 64,00', '€ 120,00', '€ 39,00', '€ 40,00', '€ 29,00', '€ 26,00',
       '€ 36,00', '€ 98,00', '€ 19,00', '€ 56,00', '€ 21,00', '€ 55,00',
       '€ 63,00', '€ 74,00', '€ 75,00', '€ 35,00', '€ 34,00', '€ 52,00',
       '€ 104,00', '€ 82,00', '€ 78,00', '€ 20,00', '€ 72,00', '€ 73,00',
       '€ 66,00', '€ 88,00', '€ 270,00', '€ 340,00', '€ 407,00',
       '€ 410,00', '€ 860,00', '€ 890,00', '€ 180,00', '€ 260,00',
       '€ 58,00', '€

In [177]:
rows_to_drop = add_to_drop(wines_original['price'] == 'PREZZI AL BICCHIERE', reason='price is "PREZZI AL BICCHIERE"')

In [178]:
wines['price'] = wines_original['price'].fillna('0')
wines['price'] = wines['price'].apply(lambda x: x.replace('.', '').replace(',', '.').replace('€', '').strip())
wines['price'] = wines['price'].apply(lambda x: int(float(x)*100) if x != 'PREZZI AL BICCHIERE' else x)
wines['price'].unique()

array([0, 8500, 8000, 2500, 1500, 2400, 1800, 2300, 13000, 2700, 4200,
       1600, 9500, 11000, 23000, 16000, 85000, 40000, 38000, 9000, 45000,
       90000, 21000, 22000, 42000, 14000, 32000, 24000, 19000, 15000,
       5000, 10500, 5400, 10000, 6000, 6500, 25000, 3800, 4500, 4800,
       3000, 2800, 7000, 3300, 6400, 12000, 3900, 4000, 2900, 2600, 3600,
       9800, 1900, 5600, 2100, 5500, 6300, 7400, 7500, 3500, 3400, 5200,
       10400, 8200, 7800, 2000, 7200, 7300, 6600, 8800, 27000, 34000,
       40700, 41000, 86000, 89000, 18000, 26000, 5800, 3700, 6800, 18500,
       14500, 3200, 17000, 84000, 12500, 33000, 13200, 8600, 20000, 36000,
       5700, 12300, 2200, 6200, 9900, 4600, 15500, 28000, 30000, 65000,
       200000, 58000, 3100, 125, 5100, 12600, 125000, 120000, 9400, 48000,
       52000, 1000, 100000, 800, 700, 600, 'PREZZI AL BICCHIERE', 500,
       1200, 1400, 900], dtype=object)

#### info

In [179]:
# add empty info
wines['info'] = ''

#### quantity

In [180]:
# parse quantity
# sorted(wines_original['quantity'].astype(str).apply(str.strip).unique())

In [181]:
# check if there are wines with multiple vintages and multiple quantities for each vintage at the same
wines.loc[(wines_original['quantity'].apply(lambda x: len(str(x))) >= 4) & (wines['vintage'].apply(lambda x: type(x)==tuple))]

,external_id,size,winery_name,country,vintage,currency,price,quantity,Unnamed: 10,name,type,storage_area,info


In [182]:
wines_original['quantity'] #[[37, 38, 39]].apply(len) == 0

0      NaN
1       23
2       19
3      NaN
4      NaN
      ... 
923    NaN
924    NaN
925    NaN
926      4
927      1
Name: quantity, Length: 928, dtype: object

In [183]:
# nice, there is none. We can safely explode the quantity and vintage columns without duplicating stuff
# We can therefore work independently on the two columns
wines['quantity'] = wines_original['quantity'].fillna('0').astype(str)

import re
PATTERN = r"^\s*\d+\s*$|\d+\s+[‘’']\d{2}"

wines['quantity'] = wines['quantity'].apply(lambda x: tuple(re.findall(PATTERN, x)))
rows_to_drop = add_to_drop(wines['quantity'].apply(len) == 0, 'quantity does not correspond to pattern')

for i, row in wines.iterrows():
    print(i, row['external_id'], row['quantity'])

0 1 ('0',)
1 2 ('23',)
2 3 ('19',)
3 4 ('0',)
4 5 ('0',)
5 6 ('0',)
6 7 ('0',)
7 8 ('0',)
8 9 ('0',)
9 10 ('0',)
10 11 ('10',)
11 12 ('0',)
12 13 ('0',)
13 14 ('4',)
14 15 ('0',)
15 16 ('0',)
16 17 ('2',)
17 18 ('1 ‘04', '2 ‘08')
18 19 ('2',)
19 20 ('3',)
20 21 ('2',)
21 22 ('13',)
22 23 ('2',)
23 24 ('83',)
24 25 ('14',)
25 26 ('10',)
26 27 ('3',)
27 28 ('0',)
28 29 ('1',)
29 30 ('1',)
30 31 ('1',)
31 32 ('2',)
32 33 ('12',)
33 34 ('2',)
34 35 ('2',)
35 36 ('1',)
36 37 ('0',)
37 38 ('0',)
38 39 ()
39 40 ('4',)
40 41 ()
41 42 ('5',)
42 43 ('1',)
43 44 ('0',)
44 45 ('0',)
45 46 ('6',)
46 47 ('6',)
47 48 ('1',)
48 49 ('5',)
49 50 ('13',)
50 51 ('9',)
51 52 ('27',)
52 53 ('6',)
53 54 ('13 ‘14',)
54 55 ('4',)
55 56 ('0',)
56 57 ('9',)
57 58 ('0',)
58 59 ('1',)
59 60 ('19',)
60 61 ('2',)
61 62 ('1',)
62 63 ('1',)
63 64 ('0',)
64 65 ('12',)
65 66 ('1',)
66 67 ('1',)
67 68 ('6',)
68 69 ('0',)
69 70 ('173',)
70 71 ('4',)
71 72 ('0',)
72 73 ('0',)
73 74 ('0',)
74 75 ('0',)
75 76 ()
76 77 ('8',)

#### Internal notes

In [184]:
# internal_notes: missing
wines['internal_notes'] = ''

### Save new version of the winelist

In [185]:
# drop columns that are not the required ones
# - `external_id`
# - `name`
# - `winery_name`
# - `type`
# - `storage_area`
# - `size`
# - `vintage`
# - `price`
# - `info`
# - `quantity`
# - `internal_notes`


# output dropped rows
open('v2-dropped.csv', 'w').close()
rows_to_drop.drop_duplicates().to_csv('v2-dropped.csv', index=False)

In [186]:
wines_out = wines[['external_id', 'name', 'winery_name', 'type', 'storage_area', 'size', 'vintage', 'price', 'info', 'quantity', 'internal_notes']]
wines_out = wines_out.drop(rows_to_drop.index)

wines_out = wines_out.explode('vintage').reset_index(drop=True)
wines_out = wines_out.explode('quantity').reset_index(drop=True)

# process quantity
for idx, row in wines_out.iterrows():
    if len(re.findall(r"\d+\s+[‘’']\d{2}", row['quantity'])) > 0:
        quantity, vintage = re.split(r"\s+[‘’']", row['quantity'])
        vintage = int(vintage)

        if vintage < 24:
            vintage = 2000 + vintage
        else:
            vintage = 1900 + vintage

        wines_out.loc[idx, 'quantity'] = int(quantity)
        wines_out.loc[idx, 'vintage'] = vintage

# for idx, row in wines_out.iterrows():
#     print(idx, row['external_id'], row['quantity'])

open('v2-cleaned.csv', 'w').close()
wines_out.drop_duplicates().to_csv('v2-cleaned.csv', index=False)

### Print search terms

**NOTE: Before proceeding with the matching algorithm, run the viviner and insert new wines.**

In [187]:
with open('search-terms.txt', 'w') as f:
    for term in wines['name'].unique():
        f.write(f'{term}\n')
    for term in wines['winery_name'].unique():
        f.write(f'{term}\n')

In [188]:
wines_out.loc[[15,16,17]]

,external_id,name,winery_name,type,storage_area,size,vintage,price,info,quantity,internal_notes
15,17,champagne cuvée brut,Laurent Perrier,,,BOTTLE,None,8000,,2,
16,18,champagne millesimato,Laurent Perrier,,,BOTTLE,2004,9500,,1,
17,18,champagne millesimato,Laurent Perrier,,,BOTTLE,2008,9500,,2,
